# 作成した2次元埋め込みベクトルを偏角10～170°にスケーリングする

## 1. ライブラリのインポート

In [35]:
import numpy as np
import os
from tqdm import tqdm

In [36]:
model_name = "esm1b"  # モデル名
method_name = "umap2"  # 使った次元削減の手法

vectors_dir = f"../data/embedding-vectors/{model_name}/{method_name}"

files = [os.path.join(vectors_dir, f) for f in os.listdir(vectors_dir) if f.endswith(".npy")]  # .npy ファイルを全て取得
files.sort()  # 一応ソート

In [37]:
def map_range(x, a, b, c, d):
    """
    変数 $x$ を 範囲 [$a$, $b$] から [$c$, $d$] にリマップします
    :param x: 変換する対象
    :param a: 変換前の下端
    :param b: 変換前の上端
    :param c: 変換後の下端
    :param d: 変換後の上端
    :return: リマップした値
    """
    return (x - a) / (b - a) * (d - c) + c

## 2. データを保存

In [38]:
output_dir = f"../data/embedding-vectors/{model_name}/{method_name}_scaled_10_170"
os.makedirs(output_dir, exist_ok=True)

In [39]:
# import matplotlib.pyplot as plt

for filename in tqdm(files):
    old_vec = np.load(filename, mmap_mode="r")
    new_vec = np.zeros(old_vec.shape)
    h = old_vec.shape[0]

    for i in range(h):
        x = old_vec[i][0]
        y = old_vec[i][1]
        z = x + 1j * y
        r = np.abs(z)
        theta = np.angle(z, deg=True)
        
        new_theta = np.radians(map_range(theta, -180, 180, 10, 170))
        new_x = r * np.cos(new_theta)
        new_y = r * np.sin(new_theta)
        new_vec[i] = [new_x, new_y]

    save_filename = os.path.basename(filename).replace(method_name, f"{method_name}_scaled_10_170")
    np.save(os.path.join(output_dir, save_filename), new_vec)

    # --- プロット部 ---
    # fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    # old_vec プロット
    # axs[0].scatter(old_vec[:, 0], old_vec[:, 1], c='blue', s=1)
    # axs[0].set_title('Old Vectors')
    # axs[0].axis('equal')

    # new_vec プロット
    # axs[1].scatter(new_vec[:, 0], new_vec[:, 1], c='red', s=1)
    # axs[1].set_title('New Vectors')
     #axs[1].axis('equal')

    # plt.tight_layout()
    # plt.show()

100%|██████████████████████████████████████████████████████████████████████████████| 7716/7716 [00:40<00:00, 191.74it/s]
